### 第8回レポート課題その2

BetterRnnlmクラスの学習を行った.

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/deep-learning-from-scratch-2-master')
from common import config

config.GPU = True

from common.optimizer import SGD
from common.trainer import RnnlmTrainer
from common.util import eval_perplexity, to_gpu
from dataset import ptb
sys.path.append('/content/drive/My Drive/deep-learning-from-scratch-2-master/ch06')
from better_rnnlm import BetterRnnlm


# ハイパーパラメータの設定
batch_size = 20
wordvec_size = 650
hidden_size = 650
time_size = 35
lr = 20.0
max_epoch = 40
max_grad = 0.25
dropout = 0.5

# 学習データの読み込み
corpus, word_to_id, id_to_word = ptb.load_data('train')
corpus_val, _, _ = ptb.load_data('val')
corpus_test, _, _ = ptb.load_data('test')

if config.GPU:
    corpus = to_gpu(corpus)
    corpus_val = to_gpu(corpus_val)
    corpus_test = to_gpu(corpus_test)

vocab_size = len(word_to_id)
xs = corpus[:-1]
ts = corpus[1:]

model = BetterRnnlm(vocab_size, wordvec_size, hidden_size, dropout)
optimizer = SGD(lr)
trainer = RnnlmTrainer(model, optimizer)

best_ppl = float('inf')
for epoch in range(max_epoch):
    trainer.fit(xs, ts, max_epoch=1, batch_size=batch_size,
                time_size=time_size, max_grad=max_grad)

    model.reset_state()
    ppl = eval_perplexity(model, corpus_val)
    print('valid perplexity: ', ppl)

    if best_ppl > ppl:
        best_ppl = ppl
        model.save_params()
    else:
        lr /= 4.0
        optimizer.lr = lr

    model.reset_state()
    print('-' * 50)


# テストデータでの評価
model.reset_state()
ppl_test = eval_perplexity(model, corpus_test)
print('test perplexity: ', ppl_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Done
Done
| epoch 1 |  iter 1 / 1327 | time 14[s] | perplexity 10000.10
| epoch 1 |  iter 21 / 1327 | time 21[s] | perplexity 3783.88
| epoch 1 |  iter 41 / 1327 | time 26[s] | perplexity 1726.94
| epoch 1 |  iter 61 / 1327 | time 32[s] | perplexity 1357.04
| epoch 1 |  iter 81 / 1327 | time 38[s] | perplexity 1026.60
| epoch 1 |  iter 101 / 1327 | time 44[s] | perplexity 852.42
| epoch 1 |  iter 121 / 1327 | time 50[s] | perplexity 826.61
| epoch 1 |  iter 141 / 1327 | time 56[s] | perplexity 741.70
| epoch 1 |  iter 161 / 1327 | time 62[s] | perplexity 695.38
| epoch 1 |  iter 181 / 1327 | time 68[s] | perplexity 704.41
| epoch 1 |  iter 201 / 1327 | time 74[s] | perplexity 588.94
| epoch 1 |  iter 221 / 1327 | time 79[s] | perplexity 585.31
| epoch 1 |  iter 241 / 1327 | time 85[s] | perplexity 516.51
| epoch 1 |  iter 261 / 1327 | time 91[s] | perplexity 

単純なRnnlmクラスの場合, 最終的なパープレキシティの値は134.86であったので, およそ半分の値まで減少させることができた.

### 感想
実行に5時間もかかるプログラムを動かしたことがなかったので半分冗談かと思っていたら, 本当にかかったので驚いた. CPUを使うと2日もかかることからGPUのありがたさを実感した.

### 参考文献
斎藤 康毅　『ゼロから作るDeep Learning②自然言語処理編』, 2018, オライリー・ジャパン, p.223-275